# 1. Load Data

### 1.1 Imports

In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
import time
from pyspark import SparkFiles
import pandas as pd

In [2]:
# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/06/08 19:44:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### 1.2 Load Data

#### Medical Conditions

In [3]:
spark.sparkContext.addFile('Resources/medical_conditions_cleaned.csv')
medical_conditions = spark.read.csv(SparkFiles.get("medical_conditions_cleaned.csv"), sep=",", header=True, inferSchema=True)
medical_conditions.show(5)

23/06/08 19:45:07 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
+--------+----------+---------------+-----------------+-------------------------+-------------------+----------------+----------+-----------+---------+-----------+--------------+---------------+---------------------+--------------------------+-------------------+------------------+---------------+-----------------+---------------------+------------+------+-------------+---------------------+---------------+------+-------+-----+--------------+----------+-----------------+-------------------------+-------------------+----------+-----------+-----------------+------------------+-------+
|      id|asthma_age|ever_overweight|heart_failure_age|age_chronic_heart_disease|angina_pectoris_age|heart_attack_age|stroke_age|thyroid_age|liver_age|fatty_liver|liver_fibrosis|liver_cirrhosis|liver_viral_hepatitis|liver_autoimmu

In [4]:
medical_conditions.createOrReplaceTempView("medical_conditions")

In [5]:
medical_conditions.printSchema()

root
 |-- id: double (nullable = true)
 |-- asthma_age: double (nullable = true)
 |-- ever_overweight: integer (nullable = true)
 |-- heart_failure_age: double (nullable = true)
 |-- age_chronic_heart_disease: double (nullable = true)
 |-- angina_pectoris_age: double (nullable = true)
 |-- heart_attack_age: double (nullable = true)
 |-- stroke_age: double (nullable = true)
 |-- thyroid_age: double (nullable = true)
 |-- liver_age: double (nullable = true)
 |-- fatty_liver: integer (nullable = true)
 |-- liver_fibrosis: integer (nullable = true)
 |-- liver_cirrhosis: integer (nullable = true)
 |-- liver_viral_hepatitis: integer (nullable = true)
 |-- liver_autoimmune_hepatitis: integer (nullable = true)
 |-- other_liver_disease: integer (nullable = true)
 |-- gallbladder_pr_age: double (nullable = true)
 |-- relative_asthma: integer (nullable = true)
 |-- relative_diabetes: integer (nullable = true)
 |-- relative_heart_attack: integer (nullable = true)
 |-- heart_attack: integer (nullab

In [6]:
spark.sql(" \
            select count(*) \
            from medical_conditions ").show()

+--------+
|count(1)|
+--------+
|   14890|
+--------+



In [7]:
spark.sql(" \
            select count(*) \
            from medical_conditions \
            where heart_attack = 1 ").show()

+--------+
|count(1)|
+--------+
|     416|
+--------+



#### Health Habits

In [8]:
spark.sparkContext.addFile('Resources/total_df2.csv')
health_habits = spark.read.csv(SparkFiles.get("total_df2.csv"), sep=",", header=True, inferSchema=True)
health_habits.show(5)

+--------+-----------------------+---------------------+--------------+----+------------+--------------+
|      id|Total_Cholesterol_mg_dL|Frequency_of_Drinking|Drinks_per_Day|Work|Recreational|100_Cigarettes|
+--------+-----------------------+---------------------+--------------+----+------------+--------------+
|109266.0|                  195.0|                 10.0|           1.0|   0|           1|             0|
|109274.0|                  105.0|                  4.0|           2.0|   1|           0|             0|
|109292.0|                  172.0|                  4.0|           6.0|   0|           0|             0|
|109297.0|                  214.0|                  6.0|           2.0|   1|           0|             0|
|109307.0|                  161.0|                  9.0|           1.0|   0|           0|             1|
+--------+-----------------------+---------------------+--------------+----+------------+--------------+
only showing top 5 rows



In [9]:
health_habits.createOrReplaceTempView("health_habits")

In [10]:
health_habits.printSchema()

root
 |-- id: double (nullable = true)
 |-- Total_Cholesterol_mg_dL: double (nullable = true)
 |-- Frequency_of_Drinking: double (nullable = true)
 |-- Drinks_per_Day: double (nullable = true)
 |-- Work: integer (nullable = true)
 |-- Recreational: integer (nullable = true)
 |-- 100_Cigarettes: integer (nullable = true)



In [11]:
spark.sql(" \
            select count(*) \
            from health_habits ").show()

+--------+
|count(1)|
+--------+
|    5502|
+--------+



#### Demographics

In [12]:
spark.sparkContext.addFile('Resources/demographics.csv')
demo = spark.read.csv(SparkFiles.get("demographics.csv"), sep=",", header=True, inferSchema=True)
demo.show(5)

+--------+----+-------------+-----------+----------+----------+---------------------+----------+-------------------+----------+
|      id| age|gender_female|gender_male|race_asian|race_black|race_mexican_american|race_other|race_other_hispanic|race_white|
+--------+----+-------------+-----------+----------+----------+---------------------+----------+-------------------+----------+
|109263.0| 2.0|            0|          1|         1|         0|                    0|         0|                  0|         0|
|109264.0|13.0|            1|          0|         0|         0|                    1|         0|                  0|         0|
|109265.0| 2.0|            0|          1|         0|         0|                    0|         0|                  0|         1|
|109266.0|29.0|            1|          0|         1|         0|                    0|         0|                  0|         0|
|109267.0|21.0|            1|          0|         0|         0|                    0|         0|        

In [13]:
demo.createOrReplaceTempView("demo")

In [14]:
demo.printSchema()

root
 |-- id: double (nullable = true)
 |-- age: double (nullable = true)
 |-- gender_female: integer (nullable = true)
 |-- gender_male: integer (nullable = true)
 |-- race_asian: integer (nullable = true)
 |-- race_black: integer (nullable = true)
 |-- race_mexican_american: integer (nullable = true)
 |-- race_other: integer (nullable = true)
 |-- race_other_hispanic: integer (nullable = true)
 |-- race_white: integer (nullable = true)



In [15]:
spark.sql(" \
            select count(*) \
            from demo ").show()

+--------+
|count(1)|
+--------+
|   15560|
+--------+



# 2. Join Tables

### 2.1 Join

In [16]:
df_full = spark.sql(" \
            select t1.*, \
                   t2.Total_Cholesterol_mg_dL, \
                   t2.Frequency_of_Drinking, \
                   t2.Drinks_per_Day, \
                   t2.Work, \
                   t2.Recreational, \
                   t2.100_Cigarettes, \
                   t3.age, \
                   t3.gender_female, \
                   t3.gender_male, \
                   t3.race_asian, \
                   t3.race_black, \
                   t3.race_mexican_american, \
                   t3.race_other, \
                   t3.race_other_hispanic, \
                   t3.race_white \
            from medical_conditions as t1 \
                join health_habits as t2 on t1.id = t2.id \
                join demo as t3 on t1.id = t3.id ")
df_full.show(5)

+--------+----------+---------------+-----------------+-------------------------+-------------------+----------------+----------+-----------+---------+-----------+--------------+---------------+---------------------+--------------------------+-------------------+------------------+---------------+-----------------+---------------------+------------+------+-------------+---------------------+---------------+------+-------+-----+--------------+----------+-----------------+-------------------------+-------------------+----------+-----------+-----------------+------------------+-------+-----------------------+---------------------+--------------+----+------------+--------------+----+-------------+-----------+----------+----------+---------------------+----------+-------------------+----------+
|      id|asthma_age|ever_overweight|heart_failure_age|age_chronic_heart_disease|angina_pectoris_age|heart_attack_age|stroke_age|thyroid_age|liver_age|fatty_liver|liver_fibrosis|liver_cirrhosis|liver

In [17]:
df_full.createOrReplaceTempView("df_full")

In [18]:
df_full.printSchema()

root
 |-- id: double (nullable = true)
 |-- asthma_age: double (nullable = true)
 |-- ever_overweight: integer (nullable = true)
 |-- heart_failure_age: double (nullable = true)
 |-- age_chronic_heart_disease: double (nullable = true)
 |-- angina_pectoris_age: double (nullable = true)
 |-- heart_attack_age: double (nullable = true)
 |-- stroke_age: double (nullable = true)
 |-- thyroid_age: double (nullable = true)
 |-- liver_age: double (nullable = true)
 |-- fatty_liver: integer (nullable = true)
 |-- liver_fibrosis: integer (nullable = true)
 |-- liver_cirrhosis: integer (nullable = true)
 |-- liver_viral_hepatitis: integer (nullable = true)
 |-- liver_autoimmune_hepatitis: integer (nullable = true)
 |-- other_liver_disease: integer (nullable = true)
 |-- gallbladder_pr_age: double (nullable = true)
 |-- relative_asthma: integer (nullable = true)
 |-- relative_diabetes: integer (nullable = true)
 |-- relative_heart_attack: integer (nullable = true)
 |-- heart_attack: integer (nullab

In [19]:
spark.sql(" \
            select count(*) \
            from df_full \
            where heart_attack = 1 ").show()

+--------+
|count(1)|
+--------+
|     195|
+--------+



In [20]:
spark.sql(" \
            select count(*) \
            from df_full \
            where heart_attack = 0 \
                and stroke_age != 0.0").show()

+--------+
|count(1)|
+--------+
|     166|
+--------+



### 2.2 Conver to .csv file

In [21]:
df_full_pandas = df_full.toPandas()
df_full_pandas.head()

,id,asthma_age,ever_overweight,heart_failure_age,age_chronic_heart_disease,angina_pectoris_age,heart_attack_age,stroke_age,thyroid_age,liver_age,...,100_Cigarettes,age,gender_female,gender_male,race_asian,race_black,race_mexican_american,race_other,race_other_hispanic,race_white
0,109266.0,0.0,1,0.0,0.0,0.0,NaN,0.0,0.0,0.0,...,0,29.0,1,0,1,0,0,0,0,0
1,109274.0,0.0,1,0.0,0.0,0.0,NaN,0.0,0.0,0.0,...,0,68.0,0,1,0,0,0,1,0,0
2,109292.0,52.0,0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,...,0,58.0,0,1,0,0,0,0,1,0
3,109297.0,0.0,0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,...,0,30.0,1,0,1,0,0,0,0,0
4,109307.0,0.0,0,0.0,0.0,42.0,NaN,0.0,0.0,0.0,...,1,47.0,0,1,1,0,0,0,0,0


In [22]:
df_full_pandas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5478 entries, 0 to 5477
Data columns (total 53 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   id                          5478 non-null   float64
 1   asthma_age                  5478 non-null   float64
 2   ever_overweight             5478 non-null   int32  
 3   heart_failure_age           5478 non-null   float64
 4   age_chronic_heart_disease   5478 non-null   float64
 5   angina_pectoris_age         5478 non-null   float64
 6   heart_attack_age            195 non-null    float64
 7   stroke_age                  5478 non-null   float64
 8   thyroid_age                 5478 non-null   float64
 9   liver_age                   5478 non-null   float64
 10  fatty_liver                 5478 non-null   int32  
 11  liver_fibrosis              5478 non-null   int32  
 12  liver_cirrhosis             5478 non-null   int32  
 13  liver_viral_hepatitis       5478 

In [23]:
df_full_pandas.to_csv('Resources/heart_attack_final_table.csv', index=False)